## Домашняя работа № 7 - ML как http-сервис

### Мацыкина А.С.

#### Т120-101М-20

### Домашнее задание : ML как http-сервис

## Задача 1: применяем PCA-трансформацию

Модифицируйте файл `train.py` - добавьте в пайплайн обучения модели сжатие размерности до `n_components=2` с помощью [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) и обучите модель **в докере** на "сжатых" данных. Сохраните полученный объект `pca_transformer.pkl`, который умеет выполнять сжатие данных.

Решением домашки считается модифицированный файл *train.py*

In [2]:
# --- ВАШ КОД ТУТ --
import pickle
import os
import logging
from pathlib import Path

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'

log_filename = "/www/classifier/data/service.log"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

# загрузка данных
def load_data(from_file: Path):
    data_source = np.genfromtxt(from_file.resolve().as_posix(), delimiter=',', skip_header=1)
    X = data_source[:, :3]
    y = data_source[:, 3]
    
    return X, y

# сжатие
def transform(X, path: Path):
    pca_transformer = PCA(n_components=2).fit(X)
    X_pca = pca_transformer.transform(X)
    

    with path.open('wb') as f:
        pickle.dump(pca_transformer, f)
        logging.info('Модель сжата с помощью PCA и сохранена в %s' % path.resolve())

    return X_pca

# обучение модели
def train(X, y, path: Path): 
    clf = DecisionTreeClassifier(max_depth=3, random_state=42)
    clf.fit(X, y)


    with path.open('wb') as f:
        pickle.dump(clf, f)
        logging.info('Модель обучена и сохранена в %s' % path.resolve())


def train_1():
    X, y = load_data(Path('data/client_segmentation.csv'))
    X = transform(X, path=Path('./data/pca_transformer.pkl'))

    train(X, y, path=Path('./data/clf.pkl'))

train_1()

# ------------------

2021-05-01 16:15:59,559 | INFO     | <ipython-input-2-806ee4f0:32   | Модель сжата с помощью PCA и сохранена в /srv/data_client/jupyter_notebooks/data/pca_transformer.pkl
2021-05-01 16:15:59,568 | INFO     | <ipython-input-2-806ee4f0:44   | Модель обучена и сохранена в /srv/data_client/jupyter_notebooks/data/clf.pkl


### Домашнее задание: трансформация входных фичей на лету

Модифицируйте файл `service.py`: добавьте загрузку объекта для трансформации `tsne_tansformer.pkl` и применяйте её **в докере** для трансформации набора входных фич в сжатые:
<pre>
[x1, x2, x3] -> [x1_tsne, x2_tsne]
</pre>

Соответственно, predict надо выполнять на *сжатых* фичах


Решением домашки считается модифицированный файл *service.py*

In [46]:
# --- ВАШ КОД ТУТ --

"""
Умеет выполнять классификацию клиентов по трём фичам

Запускаем из python3:
    python3 service.py
Проверяем работоспособность:
    curl http://127.0.0.1:5000/
"""
import json
import http.server
import logging
import os
import pickle
import socketserver
import sys
from http import HTTPStatus
from re import compile

import numpy as np
from sklearn.tree import DecisionTreeClassifier

# файл, куда посыпятся логи модели

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)


def parse_params(params) -> dict:
    """
        Выдираем параметры из GET-запроса
    """
    params_list = params.split('&')
    params_dict = {'x1': None, 'x2': None, 'x3': None}
    for param in params_list:
        key, value = param.split('=')
        params_dict[key] = float(value)
    return params_dict


class Handler(http.server.SimpleHTTPRequestHandler):
    """Простой http-сервер"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_response(self) -> dict:
        """Пример запроса
        
        http://0.0.0.0:5000/classifier/?x1=1&x2=-2.2&x3=1.05
        """
        response = {'ping': 'ok'}
        params_parsed = self.path.split('?')
        if len(params_parsed) == 2 and self.path.startswith('/classifier'):
            params = params_parsed[1]
            params_dict = parse_params(params)
            response = params_dict
            
            user_features = np.array([params_dict['x1'], params_dict['x2'], params_dict['x3']]).reshape(1, -1)
            
            reduced_features = transformer.transform(user_features)
            predicted_class = int(classifier_model.predict(user_features)[0])
            logging.info('predicted_class %s' % predicted_class)
            response.update({'predicted_class': predicted_class})
        elif self.path.startswith('/ping/'):
            response = {'message': 'pong'}

        return response

    def do_GET(self):
        # заголовки ответа
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps(self.get_response()).encode())


logging.info('Загружаем обученную модель')
with open('./clf.pkl', 'rb') as f:
    classifier_model = pickle.load(f)
    logging.info('Модель загружена: %s' % classifier_model)

with open('./pca_transformer.pkl', 'rb') as f:
    transformer = pickle.load(f)
    logging.info('Модель загружена: %s' % transformer)


# ------------------

2021-05-01 15:57:14,673 | INFO     | <ipython-input-46-d85f639:79   | Загружаем обученную модель
2021-05-01 15:57:14,683 | INFO     | <ipython-input-46-d85f639:82   | Модель загружена: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')
2021-05-01 15:57:14,691 | INFO     | <ipython-input-46-d85f639:86   | Модель загружена: PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)


### Домашнее задание: Используем Flask

Перепишите сервис на использование Flask. Вы можете взять готовый базовый образ с Flask, либо добавить установку в тот контейнер, который есть - это нужно сделать в Dockerfile


In [ ]:
# --- ВАШ КОД ТУТ --

# ------------------

## Домашнее задание: строим KNN

Загрузим исходные данные - там примерно полмиллиона просмотров:

In [1]:
import pandas as pd
import numpy as np

content_views = pd.read_csv(
    'recsys_data/content_views.zip', delimiter=',', header=0, compression='zip',
    names = ['user_id', 'content_id', 'view_duration', 'view_ts', 'dt', 'platform'],
    dtype = {'user_id': np.uint32, 'content_id': np.uint16, 'view_duration': np.uint16},
    parse_dates = [3, 4]
)


print('Количество просмотров %s' % content_views.user_id.count())

content_views.head(3)

Количество просмотров 489565


,user_id,content_id,view_duration,view_ts,dt,platform
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG


Загрузим дополнительную информацию о контенте: жанры, дату появления на сервисе, рейтинг кинопоиска и т.д.

In [2]:
content_description = pd.read_csv(
    'recsys_data/content_description.zip', delimiter=',', header=0, compression='zip',
    names = ['content_id', 'origin_country', 'release_date', 'kinopoisk_rating', 'compilation_id', 'genre'],
    dtype = {'content_id': np.uint16},
    parse_dates = [2]
)

print('Количество доступного контента %s' % content_description.content_id.count())

content_description.head(3)

Количество доступного контента 126182


,content_id,origin_country,release_date,kinopoisk_rating,compilation_id,genre
0,1974,87.0,2009-12-15,7.27,153,Для детей
1,2148,87.0,2009-12-21,7.27,153,Для детей
2,2184,87.0,2009-12-22,7.27,153,Для детей


Сформируем разреженную матрицу user-item такую, что

* количество строк матрицы совпадает с числом пользователей
* количество столбцов матрицы совпадает с количеством контента
* на пересечении столбца $i$ и строки $j$ стоит единица, если пользователь $i$ смотрел контент $j$, иначе - ноль

Для начала перейдём от индекса контента и индекса пользователя к индексам в разреженной матрице - воспользуемся `LabelEncoder`

In [3]:
from sklearn.preprocessing import LabelEncoder

# кодируем индексы пользователей
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)

# ереиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)

# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)

# нова переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)


content_views.head()

,user_id,content_id,view_duration,view_ts,dt,platform,user_index,item_index
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG,802,22812
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG,2,20399
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG,911,19628
3,4498,30191,297,2019-03-18 15:27:18+03:00,2019-03-18,LG,773,13517
4,4886,39349,302,2019-03-18 12:08:16+03:00,2019-03-18,LG,836,16959


Теперь у нас есть колонки `user_index, item_index`, которые соответствуют номерам строки и столбца соответственно в матрице user-item. Передадим полученные колонки в конструктор `csr_matrix`, чтобы получить разреженную матрицу

In [4]:
from scipy.sparse import csr_matrix

num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (np.ones(num_interactions),(content_views.user_index.values, content_views.item_index.values)),
    shape=(num_users, num_items)
)
print('sparsity: %.4f' % (num_interactions / (num_users * num_items)))

user_item

sparsity: 0.0091


<2000x27012 sparse matrix of type '<class 'numpy.float64'>'
	with 259994 stored elements in Compressed Sparse Row format>

Разделяем выборку на валидацию и контроль

In [5]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(
    np.arange(start=0, stop=user_item.shape[0], step=1, dtype=np.uint32),
    test_size=0.2
)
print(
    """
        Размер обучающей выборки %d пользователей
        Размер валидационной выборки %d пользователей
    """
    % (train_ids.size, test_ids.size)
)


        Размер обучающей выборки 1600 пользователей
        Размер валидационной выборки 400 пользователей
    


Мы видим, что в наше матрице 2000 пользователей и 27012 единиц контента, у матрицы высокая разреженность - менее 1% ненулевых элементов, остальное заполнено нулями.

Чтобы строить рекомендации по колаборативной модели, нам нужен быстрый способ поиска пользователей, у которых схожая история просмотров- наше предположение в том, что похожие пользователи имеют похожую историю просмотров. Для поиска схожих пользователей воспользуемся поиском ближайших соседей по нашей матрице `user-item`:

In [6]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# обучаемся только на тренировочной части пользователей
model_knn.fit(user_item[train_ids,:])

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

Создадим простейший класс, который умеет искать похожих по истории смотрения пользователей и выдавать рекомендации. Рекомендации - это контент, который смотрели ближашие соседи пользователя, а сам пользователь этот контент не видел

In [7]:
class ColaborativeFilteringKNNRecommender:
    def __init__(self, knn_model, user_item_matrix, num_neighbors):
        self.knn_model = knn_model
        self.user_item_matrix = user_item_matrix
        self.num_neighbors = num_neighbors
        self.top_recs = 50
    
    def make_recs(self, user_history: csr_matrix, top_recs: int):
        neighbors = model_knn.kneighbors(
            random_user_history,
            self.num_neighbors,
            return_distance=False
        )[0]
        full_recs = user_item[neighbors,:].max(axis=0)
        # рекомендации - это то, что насмотрели ближайшие соседи
        user_history_ids = user_history.nonzero()[1]
        # последовательность id того контента, который смотрели ближайшие соседи
        full_recs_ids = full_recs.nonzero()[1][:self.top_recs]
        # исключаем из рекомендаций то, что уже было у упользователя в историии
        success_recs = np.array([i for i in full_recs_ids if i in user_history_ids])
        print("Число успешных рекомендаций %d из %d" % (success_recs.size, top_recs))
        
        return np.array([i for i in full_recs_ids if i not in user_history_ids])[:10]


# объект рекомендателя
recommender = ColaborativeFilteringKNNRecommender(
    knn_model=model_knn,
    user_item_matrix=user_item,
    num_neighbors=10
)

Применяем рекомендатель для случайного пользователя

In [8]:
# пример рекомендаций для случайного пользователя
random_user_index = np.random.choice(test_ids)
random_user_history = user_item.getrow(random_user_index).reshape(1, -1)

recs = recommender.make_recs(random_user_history, top_recs=10)
print('user_index %d, history: %s' % (random_user_index, random_user_history.nonzero()[1][:10]))
print('recommendations: %s' % recs)

Число успешных рекомендаций 0 из 10
user_index 63, history: [6759 6760 6761]
recommendations: [ 0  1  2  5  6  7  8 80 92 94]


Почитайте документацию по модулю `implicit.nearest_neighbours.CosineRecommender`. Обучите KNN-рекомендатель и воспользуйтесь методом `recommend` для построения рекомендаций


В реальной жизни KNN-рекомендатель не стоит делать на основе `sklearn.neighbors.NearestNeighbors` - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является [пакет implictit](). В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки 

In [9]:
 #-- ВАШ КОД ТУТ --

from implicit.nearest_neighbours import CosineRecommender

cosine_recommender = CosineRecommender(K=10, num_threads=0)
cosine_recommender.fit(user_item[train_ids,:])

cosine_recommender.recommend(random_user_index, user_item[train_ids,:])
# ------------------


[(1418, 1.6466687341645e-311)]

## Домашнее задание: Item to Item

Решите задачу c2c рекомендаций - вызовите метод `similar_items` для  *item_id=1*

In [10]:
# -- ВАШ КОД ТУТ --

cosine_recommender.similar_items(1)

# ------------------

[(1, 1.0),
 (507, 0.05128205128205128),
 (742, 0.03125),
 (1017, 0.030289126640769135),
 (231, 0.027885569326658022),
 (995, 0.026279416561381837),
 (1008, 0.024883630089671975),
 (391, 0.02279803762937766),
 (563, 0.021432398984452372),
 (1373, 0.021035158095583564)]

### Домашнее задание: обучаем Implicit

Почитайте документацию по модулю implicit.als.AlternatingLeastSquares. Обучите ALS-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [11]:
# -- ВАШ КОД ТУТ --
from implicit.als import AlternatingLeastSquares

als_recommender = AlternatingLeastSquares()
als_recommender.fit(user_item[train_ids,:])

als_recommender.recommend(random_user_index, user_item[train_ids,:])

# -----------------

[(456, 0.9089784),
 (350, 0.5428846),
 (436, 0.105232686),
 (721, 0.09743038),
 (1448, 0.09049108),
 (960, 0.08574465),
 (496, 0.0788832),
 (1518, 0.07588614),
 (1179, 0.0719227),
 (235, 0.07109184)]

### Домашнее задание на метрики

Даны два вектора - истинная история пользователя и объекты, которые считает релеватными ваша модель

Вычислите

* precision
* recall
* precision@5


In [17]:
import numpy as np

user_interactions = [47315, 30004, 36322,  8942, 30820,  6086,  9126,   332, 16289,
       39106, 39335, 48506, 48654,  9234, 29935,  2678, 36202, 22636, 18007, 39328, 15414, 30016, 35601,
    58409, 21313,   386, 16303, 4397, 19644, 51887, 21659, 36325, 53030,  7764, 50266, 58734, 53419, 24121,
    50806, 36092,  8868, 28037, 36131, 13561, 16298, 27508, 41722, 30189, 46490,  2676, 43328, 781, 48397,
    41369, 39324, 36381, 39635, 27710, 47837, 28525, 12024, 56604, 41664, 37387, 48507, 413, 33526, 20059,
    49781, 56648, 16283, 50805, 34254, 39325, 59374, 22620,  8865, 27512, 13875, 30011,  7621,
    10544, 28076, 29716, 30054, 20490, 29466, 16852, 39363, 34250, 7024, 33541,   263, 21267, 25690, 23020,
    41368, 53414,  2681, 30201] 

user_recs = [
    50820, 27781, 36131, 50812, 36092, 12024, 59155, 30042, 15414, 19882, 21659, 27849, 39328, 34240, 2681,
    21267, 50126, 58560, 7764, 49781
]

# --- ВАШ КОД ТУТ ---

def precision_5_func(n, crossing_set, user_intercations):
    result = 0
    for x in np.nditer(crossing_set):
        result += 1/(user_interactions.index(x)+1)
    return result/n

doc = np.intersect1d(user_recs, user_interactions)

precision = len(doc)/len(user_recs)
recall = len(doc)/len(user_interactions)
precision_5 = precision_5_func(5, doc, user_interactions)

print(f'precision: {precision}', 
      f'recall: {recall}', 
      f'precision@5: {precision_5}', sep='\n')


# -------------------

precision: 0.5
recall: 0.1
precision@5: 0.05183403900280743
